In [ ]:
# 셀 2: 환경 변수 및 라이브러리 불러오기 (이전 코드와 동일)
import os
from dotenv import load_dotenv

load_dotenv(".env")  # .env에서 OPENAI_API_KEY 로드
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory

# LangChain의 메모리 모듈에서 ConversationBufferMemory 임포트
from langchain.memory import ConversationBufferMemory  # <--- 이 부분을 추가합니다.

# Redis URL 설정 (기본값은 localhost:6379, 필요시 .env 파일에 REDIS_URL 정의)
redis_url = os.getenv("REDIS_URL", "redis://localhost:6379/0")

# LLM 모델 초기화
llm = OpenAI(
    openai_api_key=OPENAI_API_KEY, temperature=0.7
)  # temperature는 창의성 조절

In [ ]:
# --- 멀티챗 구현 함수 ---


def get_conversation_chain(session_id: str):
    """
    주어진 session_id에 따라 Redis에 연결된 ConversationChain을 반환합니다.
    """
    # 1. RedisChatMessageHistory 인스턴스 생성
    redis_chat_history = RedisChatMessageHistory(
        session_id=session_id,
        url=redis_url,
    )

    # 2. ConversationBufferMemory에 RedisChatMessageHistory를 chat_memory로 전달
    #    return_messages=True로 설정하면 메시지 객체 리스트를 반환합니다.
    memory = ConversationBufferMemory(
        chat_memory=redis_chat_history,
        return_messages=True,  # LLM에 메시지 객체 형태로 전달 (더 유연함)
    )

    # 3. ConversationChain에 수정된 memory 객체 전달
    conversation = ConversationChain(
        llm=llm,
        memory=memory,  # <--- 수정된 부분입니다.
        verbose=False,  # 대화 과정을 너무 자세히 보여주지 않기 위해 False로 설정
        # 필요하다면 True로 변경하여 디버깅에 활용
    )
    return conversation


# --- 메인 대화 루프 (이하 동일) ---

print("--- Redis를 활용한 멀티챗 시뮬레이션 ---")
print("세션 변경: 'change <session_id>' (예: 'change user_a' 또는 'change session_1')")
print("종료: 'exit' 또는 'quit'")

current_session_id = "default_session"  # 초기 세션 ID
current_conversation = get_conversation_chain(current_session_id)
print(f"\n현재 활성 세션: {current_session_id}")

while True:
    try:
        user_input = input(f"[{current_session_id}] 당신의 질문: ")

        if user_input.lower() in ["exit", "quit"]:
            print("대화를 종료합니다.")
            break
        elif user_input.lower().startswith("change "):
            # 'change <session_id>' 명령 처리
            parts = user_input.split(" ")
            if len(parts) == 2:
                new_session_id = parts[1]
                if new_session_id == current_session_id:
                    print(f"이미 '{new_session_id}' 세션에 있습니다.")
                else:
                    current_session_id = new_session_id
                    current_conversation = get_conversation_chain(current_session_id)
                    print(f"\n세션이 '{current_session_id}'(으)로 변경되었습니다.")
            else:
                print("잘못된 'change' 명령입니다. 사용법: change <session_id>")
            continue  # 다음 루프

        # LLM에게 질문하고 응답 받기
        ai_response = current_conversation.predict(input=user_input)
        print(f"AI 응답: {ai_response}")

    except Exception as e:
        print(f"오류 발생: {e}")
        print("API 키를 확인하거나, Redis 서버가 실행 중인지 확인해주세요.")
        break

print("\n--- 모든 세션의 최종 대화 기록 (Redis에서 직접 확인) ---")
# 실험에 사용된 모든 세션 ID를 여기에 추가하여 실제 Redis에 저장된 내용을 확인할 수 있습니다.
# 위 예시에서 사용한 'default_session', 'user_a', 'session_1' 등을 넣어보세요.
test_session_ids = ["default_session", "user_a", "session_1", "user_b_test"]
for sid in test_session_ids:
    try:
        history_check = RedisChatMessageHistory(session_id=sid, url=redis_url)
        if history_check.messages:  # 메시지가 있는 세션만 표시
            print(f"\n--- 세션 '{sid}' 기록 ---")
            for msg in history_check.messages:
                print(f"  {msg.type.upper()}: {msg.content}")
    except Exception as e:
        print(f"세션 '{sid}' 기록을 가져오는 중 오류: {e}")

--- Redis를 활용한 멀티챗 시뮬레이션 ---
세션 변경: 'change <session_id>' (예: 'change user_a' 또는 'change session_1')
종료: 'exit' 또는 'quit'

현재 활성 세션: default_session
오류 발생: Error 10061 connecting to localhost:6379. 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다.
API 키를 확인하거나, Redis 서버가 실행 중인지 확인해주세요.

--- 모든 세션의 최종 대화 기록 (Redis에서 직접 확인) ---
세션 'default_session' 기록을 가져오는 중 오류: Error 10061 connecting to localhost:6379. 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다.
세션 'user_a' 기록을 가져오는 중 오류: Error 10061 connecting to localhost:6379. 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다.
세션 'session_1' 기록을 가져오는 중 오류: Error 10061 connecting to localhost:6379. 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다.
세션 'user_b_test' 기록을 가져오는 중 오류: Error 10061 connecting to localhost:6379. 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다.


### GPT

In [ ]:
# 셀 1: 패키지 설치 (한 번만 실행)
# !pip install langchain-community redis python-dotenv ipykernel

# 셀 2: 환경 변수 로드 & 임포트
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory

load_dotenv(".env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 셀 3: LLM 및 Redis 메모리 초기화
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.7)
memory = RedisChatMessageHistory(
    session_id="default_session", url="redis://localhost:6379/0"
)


# 셀 4: 대화 함수 정의 (수정된 부분)
def chat(input_text: str) -> str:
    # 1) Redis에서 이전 메시지 리스트 불러오기
    history = memory.messages  # ❌ get_messages()가 아니라 messages 속성 사용
    # 2) HumanMessage 추가하여 LLM 호출
    messages = history + [HumanMessage(content=input_text)]
    response = llm(messages)
    # 3) Redis에 대화 저장
    memory.add_user_message(input_text)
    memory.add_ai_message(response.content)
    return response.content

In [ ]:
# 셀 5: 멀티턴 대화 테스트
print(chat("안녕.난 웅이라고 해."))

C:\Users\computer\AppData\Local\Temp\ipykernel_10992\68044283.py:27: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm(messages)


안녕하세요, 웅이님. 만나서 반가워요! 어떻게 도와드릴까요? :)


In [25]:
print(chat("내 이름이 뭐라고?"))

죄송해요, 제가 이름을 잘못 말했네요. 사용자님의 이름은 웅이군요. 다시 한번 반가워요, 웅이님! 혼란을 드려 죄송합니다. 어떻게 도와드릴까요? :)
